In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# 1. Wczytanie danych
df = pd.read_csv('dataset-tickets-multi-lang-4-20k.csv')

In [ ]:



# 2. Łączenie kolumn subject i body w jeden tekst
df['text'] = df['subject'].fillna('') + ' ' + df['body'].fillna('')

# 3. Usuwamy wiersze z brakującym typem
df = df.dropna(subset=['type'])

# 4. Tokenizacja tekstu
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# 5. Kodowanie etykiet
le = LabelEncoder()
labels = le.fit_transform(df['type'])
labels_cat = to_categorical(labels)

# 6. Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(padded, labels_cat, test_size=0.2, random_state=42)

# 7. Budowa modelu LSTM
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(labels_cat.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 8. Trening
history = model.fit(X_train, y_train, validation_split=0.2, epochs=5, batch_size=64, verbose=1)

# 9. Ewaluacja
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.2f}")
